In [2]:
import tetgen
import pyvista as pv
import numpy as np

# Noisy cylinder

In [25]:
# Define cylinder parameters
h = 10.
radius = 1. # Base inner radius
t = 0.1 # Solid wall thickness

# Define angular resolution
n = 128
theta = np.linspace(0, 2 * np.pi, n, endpoint=False)

# Initialize cross sections
N = 50
points_list = [] # Inner
points_list_o = [] # Outer
dx = h/N

# Define modes
num_modes = 3
kmin = 2
kmax = 8
Amin = 0.001
Amax = 0.01

# Random seed
np.random.seed(42)

for i in range(N):
    r = radius
    xloc = dx*i-h/2 # Center cylinder at (0,0,0)

    for j in range(num_modes):
        # Get random number
        k = np.random.randint(kmin,kmax)
        A = np.random.uniform(Amin,Amax)

        # Define radial function r(θ)
        r += A * np.cos(k*theta) # Multimodal noise

    # Convert to Cartesian (x, y)
    y,z = r * np.cos(theta), r * np.sin(theta)
    yo,zo = (r+t) * np.cos(theta), (r+t) * np.sin(theta)

    # x positions for shape
    x = np.full_like(y, xloc)
    xo = np.full_like(yo, xloc)

    # Stack points
    slice_points = np.column_stack([x,y,z])
    slice_points_o = np.column_stack([xo,yo,zo])

    # Add to list
    points_list.append(slice_points)
    points_list_o.append(slice_points_o)
    

# Combine all points
points = np.vstack(points_list)
points_o = np.vstack(points_list_o)

# Create quad faces for the side
faces = []
faces_o = []
for layer in range(N - 1):  # loop over layers
    offset0 = layer * n
    offset1 = (layer + 1) * n
    for i in range(n):
        i0 = offset0 + i
        i1 = offset0 + (i + 1) % n
        j1 = offset1 + (i + 1) % n
        j0 = offset1 + i
        faces.append([4, i0, i1, j1, j0])
        faces_o.append([4, i0, i1, j1, j0])

# Flatten face list
faces = np.hstack(faces)
wall = pv.PolyData(points, faces)
faces_o = np.hstack(faces_o)
wall_o = pv.PolyData(points_o, faces_o)

# Inlet polygon
inlet = pv.Polygon(center=(0,0,-0.5), radius=1.0, n_sides=n)
inlet.points = points_list[0]  
inlet = inlet.triangulate()
inlet_o = pv.Polygon(center=(0,0,-0.5), radius=1.0, n_sides=n)
inlet_o.points = points_list_o[0]  
inlet_o = inlet_o.triangulate()

# Outlet polygon
outlet = pv.Polygon(center=(0,0,0.5), radius=1.0, n_sides=n)
outlet.points = points_list[-1]
outlet = outlet.triangulate()
outlet_o = pv.Polygon(center=(0,0,0.5), radius=1.0, n_sides=n)
outlet_o.points = points_list_o[-1]
outlet_o = outlet_o.triangulate()

# Solid wall inlet/outlet (annuli)
points_solid_inlet = np.vstack([points_list[-1],points_list_o[-1]])
points_solid_outlet = np.vstack([points_list[0],points_list_o[0]])
faces_solid_inlet = []
faces_solid_outlet = []
for i in range(n):
    # Indices
    a0 = i
    a1 = (i + 1) % n
    b0 = i + n
    b1 = ((i + 1) % n) + n

    # Triangles for quad face between (a0, a1, b1, b0)
    faces_solid_inlet.append([3, a0, a1, b1])
    faces_solid_inlet.append([3, a0, b1, b0])
    faces_solid_outlet.append([3, a0, a1, b1])
    faces_solid_outlet.append([3, a0, b1, b0])

faces_solid_inlet = np.array(faces_solid_inlet, dtype=np.int32).flatten()
solid_inlet = pv.PolyData(points_solid_inlet,faces_solid_inlet)
faces_solid_outlet = np.array(faces_solid_outlet, dtype=np.int32).flatten()
solid_outlet = pv.PolyData(points_solid_outlet,faces_solid_outlet)

# Label patches
inlet.cell_data['marker']=0
outlet.cell_data['marker']=1
wall.cell_data['marker']=2
wall_o.cell_data['marker']=3
solid_inlet.cell_data['marker']=4
solid_outlet.cell_data['marker']=5

# Merge everything
fluid = wall.merge(inlet).merge(outlet)
solid = wall.merge(solid_inlet).merge(solid_outlet).merge(wall_o)
combined = wall.merge(inlet).merge(outlet).merge(solid_inlet).merge(solid_outlet).merge(wall_o)

In [38]:
def mesh_geometry(geo,region_points,switches='pzq1.2Aa0.1f'):
    # Get surface and triangulate
    surf = geo.extract_surface().triangulate()
    
    # Create TetGen object
    tet = tetgen.TetGen(surf)

    # Add regions
    for pt, marker in region_points:
        tet.add_region(marker,pt,)

    # Tetrahedralize with quality and volume constraints (adjust 'a' for refinement)
    nodes, elem, attrib = tet.tetrahedralize(switches=switches)

    # Assign global IDs
    mesh = tet.grid
    mesh.cell_data["GlobalElementID"] = np.arange(mesh.n_cells, dtype=np.int32)  
    mesh.point_data["GlobalNodeID"] = np.arange(mesh.n_points, dtype=np.int32)

    return mesh, attrib

def extract_region(mesh,attrib):
    meshes = []
    for marker in np.unique(attrib):
        reg_mask = attrib[:,0] == marker
        region = mesh.extract_cells(reg_mask)
        meshes.append(region)
    return meshes

def split_surfaces(mesh):

    # Get the surface from mesh
    surf = mesh.extract_surface(pass_cellid=True,pass_pointid=True)

    # Isolate inlet/outlet/walls
    surf_n = surf.compute_normals(point_normals=True, cell_normals=True, inplace=False)
    arrows = surf_n.glyph(orient='Normals', scale=False, factor=0.1)
    normals = surf_n.cell_data['Normals']
    dot_threshold = 0.999  # cosine of angle (close to 1 = nearly aligned)
    axis = np.array([1.0, 0.0, 0.0]) 
    dot = normals @ axis
    inlet_ids = np.where(dot < -dot_threshold)[0]
    outlet_ids = np.where(dot > dot_threshold)[0]
    wall_ids = np.where(np.abs(dot) <= dot_threshold)[0]
    inlet_surf = surf_n.extract_cells(inlet_ids)
    outlet_surf = surf_n.extract_cells(outlet_ids)
    wall_surf = surf_n.extract_cells(wall_ids)

    # Separate walls if multiple (e.g., inner and outer walls)
    wall_surfs = wall_surf.connectivity().split_bodies()
    return inlet_surf, outlet_surf, wall_surfs

In [47]:
# Define region seed points
region_points = [
    ([0, 0, 0], 1),  # inside fluid
    ([0, 0, radius+t], 2),  # inside solid
]

# Generate mesh
mesh, attrib = mesh_geometry(combined,region_points)
meshes = extract_region(mesh,attrib)
fluid_mesh = meshes[0]
solid_mesh = meshes[1]

# Split surfaces
inlet_surf_f, outlet_surf_f, wall_surfs_f = split_surfaces(fluid_mesh)
inlet_surf_s, outlet_surf_s, wall_surfs_s = split_surfaces(solid_mesh)

# Save the volume and surface meshes
mesh_path = "/home/dana/Documents/SimVascular/svMultiPhysics/NoisyPipe/mesh/"
fluid_mesh.save(mesh_path + "fluid/mesh-complete.vtu")
inlet_surf_f.extract_surface(pass_cellid=True,pass_pointid=True).save(mesh_path + "fluid/mesh-surfaces/inlet.vtp")
outlet_surf_f.extract_surface(pass_cellid=True,pass_pointid=True).save(mesh_path + "fluid/mesh-surfaces/outlet.vtp")
wall_surfs_f[0].extract_surface(pass_cellid=True,pass_pointid=True).save(mesh_path + "fluid/mesh-surfaces/interface.vtp")
solid_mesh.save(mesh_path + "solid/mesh-complete.vtu")
inlet_surf_s.extract_surface(pass_cellid=True,pass_pointid=True).save(mesh_path + "solid/mesh-surfaces/inlet.vtp")
outlet_surf_s.extract_surface(pass_cellid=True,pass_pointid=True).save(mesh_path + "solid/mesh-surfaces/outlet.vtp")
wall_surfs_s[1].extract_surface(pass_cellid=True,pass_pointid=True).save(mesh_path + "solid/mesh-surfaces/outside.vtp")
wall_surfs_s[0].extract_surface(pass_cellid=True,pass_pointid=True).save(mesh_path + "solid/mesh-surfaces/interface.vtp")
# wall_surfs_f[0].extract_surface(pass_cellid=True,pass_pointid=True).save(mesh_path + "solid/mesh-surfaces/interface.vtp")

Delaunizing vertices...
Delaunay seconds:  0.038416
Creating surface mesh ...
Surface mesh seconds:  0.008298
Recovering boundaries...
Boundary recovery seconds:  0.050464
Removing exterior tetrahedra ...
Spreading region attributes.
Exterior tets removal seconds:  0.014294
Recovering Delaunayness...
Delaunay recovery seconds:  0.017671
Refining mesh...
  17062 insertions, added 15667 points, 1722 segments in queue.
  5681 insertions, added 5260 points, 1331 segments in queue.
  7573 insertions, added 6983 points, 859 segments in queue.
  10095 insertions, added 5074 points, 7568 subfaces in queue.
  13456 insertions, added 5209 points, 909 subfaces in queue.
  17937 insertions, added 10964 points, 777009 tetrahedra in queue.
  23910 insertions, added 14785 points, 1289874 tetrahedra in queue.
  31872 insertions, added 15831 points, 1687094 tetrahedra in queue.
  42486 insertions, added 15716 points, 1943945 tetrahedra in queue.
  56633 insertions, added 16638 points, 2098475 tetrahedr

In [48]:
# Load back and visualize
mesh_path = "/home/dana/Documents/SimVascular/svMultiPhysics/NoisyPipe/mesh/"

# Load fluid
f_vol = pv.read(mesh_path + "fluid/mesh-complete.vtu")
f_inlet = pv.read(mesh_path + "fluid/mesh-surfaces/inlet.vtp")
f_outlet = pv.read(mesh_path + "fluid/mesh-surfaces/outlet.vtp")
f_interface = pv.read(mesh_path + "fluid/mesh-surfaces/interface.vtp")
f_vol_c = f_vol.clip(normal='y',origin=(0,0,0),invert=True)
f_inlet = f_inlet.clip(normal='y',origin=(0,0,0),invert=True)
f_outlet = f_outlet.clip(normal='y',origin=(0,0,0),invert=True)
f_interface = f_interface.clip(normal='y',origin=(0,0,0),invert=True)

# Load solid
s_vol = pv.read(mesh_path + "solid/mesh-complete.vtu")
s_inlet = pv.read(mesh_path + "solid/mesh-surfaces/inlet.vtp")
s_outlet = pv.read(mesh_path + "solid/mesh-surfaces/outlet.vtp")
s_interface = pv.read(mesh_path + "solid/mesh-surfaces/interface.vtp")
s_outside = pv.read(mesh_path + "solid/mesh-surfaces/outside.vtp")
s_vol_c = s_vol.clip(normal='y',origin=(0,0,0),invert=True)
s_inlet = s_inlet.clip(normal='y',origin=(0,0,0),invert=True)
s_outlet = s_outlet.clip(normal='y',origin=(0,0,0),invert=True)
s_interface = s_interface.clip(normal='y',origin=(0,0,0),invert=True)
s_outside = s_outside.clip(normal='y',origin=(0,0,0),invert=True)

# Some camera stuff
cam = [(13,3,3),(3,0,0),(0,0,1)]
cam2 = [(0,2,0),(0,0,1),(0,0,1)]

plotter = pv.Plotter(shape=(1,2))
# Volume mesh
plotter.subplot(0,0)
plotter.camera_position = cam
plotter.add_mesh(f_vol_c, color='cyan', edge_opacity=0.5, show_edges=True) # Fluid
plotter.add_mesh(s_vol_c, color='magenta', edge_opacity=0.5, show_edges=True) # Solid
plotter.add_text("Volume Mesh", font_size=12)
plotter.add_axes()
# Surface meshes
plotter.subplot(0,1)
plotter.camera_position = cam
# Fluid
plotter.add_mesh(f_inlet, color='cyan', opacity=0.5, edge_opacity=0.5, show_edges=True)
plotter.add_mesh(f_outlet, color='magenta', opacity=0.5, edge_opacity=0.5, show_edges=True)
plotter.add_mesh(f_interface, color='green', edge_opacity=0.5, show_edges=True)
# Solid
plotter.add_mesh(s_inlet, color='cyan', edge_opacity=0.5, show_edges=True)
plotter.add_mesh(s_outlet, color='magenta', edge_opacity=0.5, show_edges=True)
plotter.add_mesh(s_outside, color='red', edge_opacity=0.5, show_edges=True)
plotter.add_text("Surface Mesh", font_size=12)
plotter.add_axes()
plotter.show()

plotter = pv.Plotter(shape=(1,2))
# Whole geometries
plotter.subplot(0,0)
plotter.camera_position = cam2
plotter.add_mesh(f_vol, color='cyan', opacity=0.5, show_edges=False) # Fluid
plotter.add_mesh(s_vol, color='magenta', opacity=0.5, show_edges=False) # Solid
plotter.subplot(0,1)
plotter.add_mesh(f_vol, color='cyan', opacity=0.5, show_edges=False) # Fluid
plotter.add_mesh(s_vol, color='magenta', opacity=0.5, show_edges=False) # Solid
plotter.show()

Widget(value='<iframe src="http://localhost:38223/index.html?ui=P_0x7bba78a65a90_14&reconnect=auto" class="pyv…

Widget(value='<iframe src="http://localhost:38223/index.html?ui=P_0x7bba230243e0_15&reconnect=auto" class="pyv…